# Collision Avoidance - Data Collection

### Stop robot if network disconnects

### Create gamepad controller

In [3]:
import os
import traitlets
import uuid
import subprocess

from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Heartbeat

import ipywidgets.widgets as widgets

#monitor wireless connection
camera_links = set()
cont_links = set()
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        print('heartbeat connection lost')
        while camera_links: camera_links.pop().unlink()
        while cont_links: cont_links.pop().unlink()
        robot.stop()

heartbeat = Heartbeat(period=5)
heartbeat.observe(handle_heartbeat_status, names='status') # attach the callback function to heartbeat status

#start controller
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

#xbox cont mapping
#http://html5gamepad.com
cont_A = 0; cont_B = 1; cont_X = 2; cont_Y = 3
cont_L1 = 4; cont_R1 = 5; cont_L2 = 6; cont_R2 = 7
cont_select = 8; cont_start = 9 
cont_L3 = 10; cont_R3 = 11
cont_up = 12; cont_down = 13; cont_left = 14; cont_right = 15
joyleft_lr = 0; joyleft_ud = 1; joyright_lr = 2; joyright_ud = 3

#hit any cont button to activate

Controller()

### Connect gamepad controller to robot motors

### Create camera instance
- Create and display Image widget
- Connect Camera to Image widget
- Display live camera feed

### Save snapshots and tag images with gamepad button

In [5]:
####### Teleoperations ########
robot = Robot()

fwd_ax = joyleft_ud
turn_ax = joyright_lr
fwd_tq = 0.35
turn_tq = 0.35
imbal = 1.05

#clear existing links
try:
    while cont_links: links.pop().unlink()
except: pass
        
#build links
cont_links = [
    #left stick fwd back right stick turn
    traitlets.dlink((controller.axes[fwd_ax], 'value'), (robot.left_motor, 'value'), transform=lambda x: -fwd_tq*imbal*x),
    traitlets.dlink((controller.axes[fwd_ax], 'value'), (robot.right_motor, 'value'), transform=lambda x: -fwd_tq*x),
    traitlets.dlink((controller.axes[turn_ax], 'value'), (robot.left_motor, 'value'), transform=lambda x: turn_tq*x),
    traitlets.dlink((controller.axes[turn_ax], 'value'), (robot.right_motor, 'value'), transform=lambda x: -turn_tq*x)
]

print("cont_links",len(cont_links))

#add button functions
for b in [cont_select,cont_start]: 
    controller.buttons[b].unobserve_all()
    controller.buttons[b].observe(lambda data: robot.stop(), names='value')

#robot.left(0.3) #hit select or start to stop
#print('hit select or start to stop spinning')

####### Camera Setup #######

camera = Camera.instance(fps=15)
active_image_view = widgets.Image(format='jpeg', width=300, height=300)
snapshot_image_view = widgets.Image(format='jpeg', width=300, height=300)

#setup snapshot function
subprocess.call(['mkdir', '-p', 'snapshots'])
def cont_save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(active_image_view.value)
        # display snapshot that was saved
        snapshot_image_view.value = active_image_view.value

#clear existing links
try:
    while camera_links: camera_links.pop().unlink()
except: pass

camera_links = [traitlets.dlink((camera, 'value'), (active_image_view, 'value'), transform=bgr8_to_jpeg)]
print("camera_links",len(camera_links))

#add button functions
for b in [cont_L1,cont_L2,cont_R1,cont_R2]: 
    controller.buttons[b].unobserve_all(); controller.buttons[b].observe(cont_save_snapshot, names='value')


####### Image Tagging #######

tagged_image_view = widgets.Image(format='jpeg', width=300, height=300)

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError: print('Directories not created becasue they already exist')
    
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='A: add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='B: add blocked', button_style='danger', layout=button_layout)

free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

active_box = widgets.VBox([widgets.Label('Active View'),active_image_view])
tagged_box = (widgets.VBox([widgets.Label('Last Tagged'),tagged_image_view]))
snap_box = (widgets.VBox([widgets.Label('Last Snapshot'),snapshot_image_view]))

display(widgets.HBox([active_box,tagged_box,snap_box]))
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

def tag_image(directory):
    image_path = os.path.join(directory, str(uuid.uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(active_image_view.value)
        tagged_image_view.value = active_image_view.value

def save_free():
    global free_dir, free_count
    tag_image(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def cont_save_free(change):
    if change['new']:
        save_free()
    
def save_blocked():
    global blocked_dir, blocked_count
    tag_image(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))

def cont_save_blocked(change):
    if change['new']:
        save_blocked()
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

#add button functions
controller.buttons[cont_A].unobserve_all()
controller.buttons[cont_B].unobserve_all() 

controller.buttons[cont_A].observe(cont_save_free, names='value')
controller.buttons[cont_B].observe(cont_save_blocked, names='value')

#controller.buttons[cont_A].unobserve_all()
#controller.buttons[cont_B].unobserve_all() 

cont_links 4


RuntimeError: Could not initialize camera.  Please see error trace.

Zip Files
---

In [1]:
!zip -r -q dataset.zip dataset